In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.svm import SVR
from scipy.stats import uniform, randint

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [12]:
df = pd.read_csv('Данные для построения модели, задача - Регрессия для SI.csv')
df

,Chi0n,SlogP_VSA2,FpDensityMorgan3,MaxAbsPartialCharge,HallKierAlpha,EState_VSA9,BCUT2D_MRHI,MinAbsEStateIndex,Target feature,SMR_VSA4,...,fr_unbrch_alkane,VSA_EState8,MolLogP,MinAbsPartialCharge,NumValenceElectrons,SMR_VSA5,Chi1v,PEOE_VSA13,VSA_EState1,Target Feature
0,19.534409,24.512883,1.321429,0.293526,-0.66,9.984809,5.944519,0.387225,1,43.480583,...,3,16.981087,7.1212,0.038844,158,105.750639,12.204226,0.0,0.000000,1.449093
1,19.794682,25.173194,1.285714,0.313407,-0.08,0.000000,5.134527,0.533868,0,33.495774,...,3,17.670565,6.1556,0.012887,162,117.834321,12.595754,0.0,0.000000,0.845098
2,23.689110,62.329944,1.156250,0.325573,-0.08,0.000000,5.117187,0.543231,0,33.495774,...,3,18.287216,7.1292,0.094802,186,117.834321,14.249005,0.0,2.517630,-0.142668
3,20.241516,24.512883,1.310345,0.293526,-0.66,9.984809,5.944502,0.390603,1,43.480583,...,4,17.012013,7.5113,0.038844,164,112.171461,12.704226,0.0,0.000000,1.800960
4,22.617677,11.423370,1.257143,0.257239,-2.22,9.984809,5.963448,0.270476,0,43.480583,...,0,14.692318,9.1148,0.062897,184,86.488175,13.868825,0.0,0.000000,0.113943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,18.825334,25.017713,2.533333,0.468587,-1.65,9.473726,5.980114,0.048029,0,51.752408,...,0,8.824371,4.3002,0.317890,164,66.219879,11.665192,0.0,10.503509,0.052701
997,21.810933,36.966734,2.457143,0.467493,-2.18,9.473726,5.980828,0.030329,0,51.752408,...,0,10.378794,3.8049,0.327562,192,79.185457,13.274017,0.0,10.086396,0.026322
998,23.633394,48.975357,2.552632,0.467485,-1.83,9.473726,7.980998,0.026535,0,51.752408,...,0,8.774745,4.5381,0.327887,210,78.682541,15.695685,0.0,10.305031,0.040112
999,23.380977,58.099656,1.864865,0.468755,-2.45,18.947452,5.950258,0.146522,0,51.752408,...,0,7.488627,3.3649,0.312509,208,66.219879,13.174959,0.0,20.885832,0.013373


In [13]:
X, y = df.drop('Target Feature', axis=1), df['Target Feature']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Target Feature', axis=1), df['Target Feature'], test_size=0.3, random_state=42)

#### Контрольной метрика для задачи регрессии будем считать R^2 score. Она будет записана в атрибут assessment_criterion BestRegressionModel класса

In [15]:
class BestRegressionModel:
    """
    Класс для отбора лучшей модели машинного обучения для модели регресии
    X: DataFrame с признаками
    y: Series с целевой переменной

    В данном классе идет подбор 4 разных моделей машинного обучения.
    Для каждой модели производиться подор гипперпараметров (если таковые имеются)
    После получения моделей с лучшими параметрами идет отбор лучшей модели по средству сравнения assessment_criterion с полученными метриками моделей
    """
    def __init__(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        self.assessment_criterion = {
            'metric_name': 'R^2',
            'value': 0.5
        }
        self.best_model = None
        self.models = {
            'linear_regression': {
                'model': LinearRegression(),
                'best_model': None,
                'best_params': None,
                'params': {},
                'parameter_selection_method': 'GridSearchCV',
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'random_forest_regressor': {
                'model': RandomForestRegressor(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'RandomizedSearchCV',
                'params': {
                    'n_estimators': randint(50, 200),
                    'max_depth': [None, 5, 10, 20],
                    'min_samples_split': randint(2, 10),
                    'min_samples_leaf': randint(1, 5)
                },
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': False,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'elastic_net': {
                'model': ElasticNet(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'GridSearchCV',
                'params': {
                    'alpha': [0.01, 0.1, 1, 10],
                    'l1_ratio': [0.2, 0.5, 0.8]
                },
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'randomized_search_cv_model': {
                'model': SVR(),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': uniform(0.1, 10),
                    'kernel': ['linear', 'rbf'],
                    'gamma': ['scale', 'auto']
                },
                'parameter_selection_method': 'RandomizedSearchCV',
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                    'y_pred': None
                }
            }
        }
        
    
    def fit(self):
        """
        Метод подбора гипперпараметров и инизиализации лучших моделей с подобранными гипперпараметрами для каждой модели
        """
        for model in self.models:
            print(f"Подбор гипперпараметров для модели {str(self.models[model]['model'])} методом {self.models[model]['parameter_selection_method']}")
            
            X, y = self.models[model]['data']['X_train'], self.models[model]['data']['y_train']
            
            if self.models[model]['standartization']:
                print('Перед подобром, необходом провести стандартизацию параметров')
                X = self.X_train_stnd = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаем к подбору гиппепараметров')
            print('Ожидайте...')
            
            if self.models[model]['parameter_selection_method'] == 'GridSearchCV':
                grid = GridSearchCV(self.models[model]['model'], self.models[model]['params'], cv=5, scoring='neg_mean_squared_error')
                grid.fit(X, y)
                self.models[model]['best_params'] = grid.best_params_
                self.models[model]['best_model'] = grid.best_estimator_
            elif self.models[model]['parameter_selection_method'] == 'RandomizedSearchCV':
                search = RandomizedSearchCV(self.models[model]['model'], self.models[model]['params'], n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42)
                search.fit(X, y)
                self.models[model]['best_params'] = search.best_params_
                self.models[model]['best_model'] = search.best_estimator_
            print(f"Лучшие параметры для модели {str(self.models[model]['model'])}:", self.models[model]['best_params'])
            print()
                
    def get_best_model(self):
        """
        Метод получения лучшей модели по показателю assessment_criterion
        В данном методе происходит обучение моделей на тестовой выборке и получение метрик на основе полученных предсказаний
        
        По итогу работы модели мы получаем модель, которая имеет лучший показатель по критрию заданному в assessment_criterion или
        уведомление о том, что модель не была выбрана. Тогда необходимо либо изменить метрику, либо изменить значение метрики
        """
        for model in self.models:
            print(f"Обучаем модель {str(self.models[model]['model'])} с лучшими гипперпараметрами для модели:")
            
            X, y = self.models[model]['data']['X_test'], self.models[model]['data']['y_test']
            
            if self.models[model]['standartization']:
                print('Перед обучением, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
            
            print('Приступаем к формированию предсказания на тестовой выборке')
            self.models[model]['data']['y_pred'] = self.models[model]['best_model'].predict(X)
            self.models[model]['metrics']['MSE'] = mean_squared_error(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['MAE'] = mean_absolute_error(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['R^2'] = r2_score(y, self.models[model]['data']['y_pred'])
            print(f"Метрики для модели {str(self.models[model]['model'])}:")
            print(f"MSE: {self.models[model]['metrics']['MSE']:.4f}")
            print(f"MAE: {self.models[model]['metrics']['MAE']:.4f}")
            print(f"R^2: {self.models[model]['metrics']['R^2']:.4f}")
            print()
            if self.models[model]['metrics'][self.assessment_criterion['metric_name']] > self.assessment_criterion['value']:
                self.assessment_criterion['value'] = self.models[model]['metrics'][self.assessment_criterion['metric_name']]
                self.best_model = self.models[model]
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована')
            print('Измените value или metric_name в assessment_criterion')
        else:
            print('Модель для формирования прогноза сформирована')
            print(f"Лучшая модель по главному критерию оценки {self.assessment_criterion['metric_name']}:")
            print(f"{str(self.best_model['best_model'])}:")
            print('Метрики')
            for k, v in self.best_model['metrics'].items():
                print(f'{k} {v:.4f}')
            
    
    def predict(self, data):
        """
        Метод для формирования прогноза с помощью лучшей модели, которая была получена в методе get_best_model
        """
        result = None
        X = data
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована, используйте метод get_best_model')
            return result
        else:
            print(f"Формируем предсказание с помощью модели {str(self.best_model['best_model'])}")
            if self.best_model['standartization']:
                print('Перед формированияем прогноза, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(data) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаю формированию прогноза...')
            result = self.best_model['best_model'].predict(X)
            print('Предсказание сформировано')
        return 10 ** result

#### Подбор гипперпараметров для моделей

In [16]:
best_regression_model = BestRegressionModel(X, y)
best_regression_model.fit()

Подбор гипперпараметров для модели LinearRegression() методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели LinearRegression(): {}

Подбор гипперпараметров для модели RandomForestRegressor(random_state=42) методом RandomizedSearchCV
Ожидайте...
Лучшие параметры для модели RandomForestRegressor(random_state=42): {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 6, 'n_estimators': 64}

Подбор гипперпараметров для модели ElasticNet(random_state=42) методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели ElasticNet(random_state=42): {'alpha': 0.01, 'l1_ratio': 0.8}

Подбор гипперпараметров для модели SVR() методом RandomizedSearchCV
Перед подобром, необходом провести стандартизацию парамет

#### Поиск лучшей модели

In [17]:
best_regression_model.get_best_model()

Обучаем модель LinearRegression() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LinearRegression():
MSE: 0.2222
MAE: 0.3336
R^2: 0.6283

Обучаем модель RandomForestRegressor(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestRegressor(random_state=42):
MSE: 0.1760
MAE: 0.3019
R^2: 0.7055

Обучаем модель ElasticNet(random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели ElasticNet(random_state=42):
MSE: 0.2132
MAE: 0.3272
R^2: 0.6434

Обучаем модель SVR() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандарт

#### Формирование предсказания лучшей моделью

In [18]:
best_regression_model.predict(X_test)

Формируем предсказание с помощью модели RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=6,
                      n_estimators=64, random_state=42)
Предсказание сформировано


array([  1.54506202,   1.66393747,  18.63114544,   1.13838924,
         2.12510479,   3.05416689,   1.80942246,   1.9964955 ,
       213.16525347,   1.57897178,   2.00324843,   1.59713281,
        22.01527228,  32.70452356,   2.02359268,  40.56464315,
         2.21969544, 137.67100911,   1.5679683 ,  44.69086582,
         2.14779899,  40.44203334,  54.58236538,   1.66512635,
         1.96912136,  27.50108474,   2.10189088,   1.74732804,
        21.22667135, 213.16525347,  22.26882757,  40.59113968,
         2.28780594,   2.19360371,   3.06183912,   2.6690685 ,
         2.18546401,   2.05657502,  22.93545077,  46.79643852,
         2.77414331,  46.79643852,   2.3463395 ,   1.07886747,
         2.87801561,   1.85178113,   2.53931066,  24.70795407,
         2.91699512,   2.27449285,  28.5433717 ,  43.3775604 ,
         1.67903575,   1.74129511,   1.59649103,   1.29799805,
         1.72949176,   2.41097648,   2.42146964,   1.03452607,
         1.82222056,  30.37682611,   2.26988851,   3.50